# Outdated

In [1]:
import numpy as np

from onnx import TensorProto, helper

import finn.core.onnx_exec as oxe
from finn.core.datatype import DataType
from finn.core.modelwrapper import ModelWrapper
from finn.transformation.fpgadataflow.prepare_ip import PrepareIP
from finn.transformation.fpgadataflow.prepare_cppsim import PrepareCppSim
from finn.transformation.fpgadataflow.compile_cppsim import CompileCppSim
from finn.transformation.fpgadataflow.hlssynth_ip import HLSSynthIP
from finn.transformation.fpgadataflow.set_exec_mode import SetExecMode
from finn.transformation.fpgadataflow.prepare_rtlsim import PrepareRTLSim
from finn.transformation.general import GiveUniqueNodeNames
from finn.util.basic import gen_finn_dt_tensor

from finn.custom_op.registry import getCustomOp
from finn.analysis.fpgadataflow.exp_cycles_per_layer import exp_cycles_per_layer
from finn.custom_op.general.im2col import compute_conv_output_dim


def make_single_im2col_modelwrapper(
    k_h, k_w, ifm_ch, ifm_dim_h, ifm_dim_w, ofm_dim_h, ofm_dim_w,
    simd, stride_h, stride_w, dilation, idt
):
    odt = idt
    inp = helper.make_tensor_value_info(
        "inp", TensorProto.FLOAT, [1, ifm_dim_h, ifm_dim_w, ifm_ch]
    )
    outp = helper.make_tensor_value_info(
        "outp", TensorProto.FLOAT, [1, ofm_dim_h, ofm_dim_w, k_h * k_w * ifm_ch]
    )

    im2col_node = helper.make_node(
        "Im2Col",
        ["inp"],
        ["outp"],
        domain="finn.custom_op.general",
        backend="fpgadataflow",
        stride=[stride_h, stride_w],
        kernel_size=[k_h, k_w],
        input_shape=str((1, ifm_dim_h, ifm_dim_w, ifm_ch)),
        dilations=dilation,
        pad_amount=[0, 0, 0, 0],
        pad_value=0,
    )
    graph = helper.make_graph(
        nodes=[im2col_node], name="im2col_graph", inputs=[inp], outputs=[outp]
    )

    model = helper.make_model(graph, producer_name="im2col-model")
    model = ModelWrapper(model)

    model.set_tensor_datatype("inp", idt)
    model.set_tensor_datatype("outp", odt)

    return model


def make_single_slidingwindow_modelwrapper(
    k_h, k_w, ifm_ch, ifm_dim_h, ifm_dim_w, ofm_dim_h, ofm_dim_w, simd,
    stride_h, stride_w, dilation, idt, dw=0
):
    odt = idt
    inp = helper.make_tensor_value_info(
        "inp", TensorProto.FLOAT, [1, ifm_dim_h, ifm_dim_w, ifm_ch]
    )
    outp = helper.make_tensor_value_info(
        "outp", TensorProto.FLOAT, [1, ofm_dim_h, ofm_dim_w, k_h * k_w * ifm_ch]
    )

    SlidingWindow_node = helper.make_node(
        "ConvolutionInputGenerator",
        ["inp"],
        ["outp"],
        domain="finn.custom_op.fpgadataflow",
        backend="fpgadataflow",
        ConvKernelDim=[k_h, k_w],
        IFMChannels=ifm_ch,
        IFMDim=[ifm_dim_h, ifm_dim_w],
        OFMDim=[ofm_dim_h, ofm_dim_w],
        SIMD=simd,
        Stride=[stride_h, stride_w],
        Dilation=[dilation, dilation],
        inputDataType=idt.name,
        outputDataType=odt.name,
        depthwise=dw,
    )
    graph = helper.make_graph(
        nodes=[SlidingWindow_node],
        name="slidingwindow_graph",
        inputs=[inp],
        outputs=[outp],
    )

    model = helper.make_model(graph, producer_name="slidingwindow-model")
    model = ModelWrapper(model)

    model.set_tensor_datatype("inp", idt)
    model.set_tensor_datatype("outp", odt)

    return model


def prepare_inputs(input_tensor):
    return {"inp": input_tensor}


def test_fpgadataflow_slidingwindow(
    idt, k_h, k_w, ifm_dim_h, ifm_dim_w, ifm_ch, stride_h, stride_w, dilation, exec_mode, simd, dw
):
    ofm_dim_h = compute_conv_output_dim(ifm_dim_h, k_h, stride_h, 0, dilation)
    ofm_dim_w = compute_conv_output_dim(ifm_dim_w, k_w, stride_w, 0, dilation)

    #x = gen_finn_dt_tensor(idt, (1, ifm_dim_h, ifm_dim_w, ifm_ch))
    x = np.arange(0, ifm_dim_h*ifm_dim_w*ifm_ch, 1)
    x = np.reshape(x, (1, ifm_dim_h, ifm_dim_w, ifm_ch)).astype(np.float32)
    print(x)
    
    model = make_single_slidingwindow_modelwrapper(
        k_h = k_h, k_w = k_w, ifm_ch = ifm_ch, ifm_dim_h = ifm_dim_h, ifm_dim_w = ifm_dim_w,
        ofm_dim_h = ofm_dim_h, ofm_dim_w = ofm_dim_w, simd = simd, stride_h = stride_h,
        stride_w = stride_w, dilation=dilation, idt=idt, dw=dw
    )
    
    model.save("/tmp/test_convInputGenerator.onnx")

    if exec_mode == "cppsim":
        model = model.transform(SetExecMode("cppsim"))
        model = model.transform(PrepareCppSim())
        model = model.transform(CompileCppSim())
        model.save("/tmp/test_convInputGenerator_compiled.onnx")
    elif exec_mode == "rtlsim":
        model = model.transform(SetExecMode("rtlsim"))
        model = model.transform(GiveUniqueNodeNames())
        model = model.transform(PrepareIP("xc7z020clg400-1", 5))
        model = model.transform(HLSSynthIP())
        model = model.transform(PrepareRTLSim())
    else:
        raise Exception("Unknown exec_mode in test_fpgadataflow_slidingwindow")

    # prepare input data
    input_dict = prepare_inputs(x)
    # execute model
    y_produced = oxe.execute_onnx(model, input_dict)["outp"]
    golden = make_single_im2col_modelwrapper(
        k_h = k_h, k_w = k_w, ifm_ch = ifm_ch, ifm_dim_h = ifm_dim_h, ifm_dim_w = ifm_dim_w,
        ofm_dim_h = ofm_dim_h, ofm_dim_w = ofm_dim_w, simd = simd, stride_h = stride_h,
        stride_w = stride_w, dilation = dilation, idt = idt
    )
    golden.save("/tmp/test_convInputGenerator_im2col.onnx")
    y_expected = oxe.execute_onnx(golden, input_dict)["outp"]
    
    print(y_produced)
    print("---")
    print(y_expected)
    if dw == 0:
        assert (y_produced == y_expected).all()
    else:
        y_expected = y_expected.reshape(
            1, ofm_dim_h, ofm_dim_w, k_h * k_w, ifm_ch // simd, simd
        )
        y_expected = y_expected.transpose(0, 1, 2, 4, 3, 5)
        y_expected = y_expected.reshape(1, ofm_dim_h, ofm_dim_w, ifm_ch * k_h * k_w)        
        assert (y_produced == y_expected).all()

    if exec_mode == "rtlsim":
        node = model.get_nodes_by_op_type("ConvolutionInputGenerator")[0]
        inst = getCustomOp(node)
        cycles_rtlsim = inst.get_nodeattr("cycles_rtlsim")
        exp_cycles_dict = model.analysis(exp_cycles_per_layer)
        exp_cycles = exp_cycles_dict[node.name]
        assert np.isclose(exp_cycles, cycles_rtlsim, atol=10)
        assert exp_cycles != 0
        

In [2]:
#test_fpgadataflow_slidingwindow(DataType.BIPOLAR, k_h=2, k_w=2, ifm_dim_h=6, ifm_dim_w=6, ifm_ch=2,
#                                stride_h=1, stride_w=1, dilation=1, exec_mode="cppsim", simd=1, dw=0)

ifm_dim_h = 6
ifm_dim_w = 6
k_h = 3
k_w = 2
stride_h = 1
stride_w = 1

ifm_ch=1
dilation=1
exec_mode="cppsim"
simd=1
dw=0



test_fpgadataflow_slidingwindow(DataType.INT32, k_h, k_w, ifm_dim_h, ifm_dim_w, ifm_ch,
                                stride_h, stride_w, dilation, exec_mode, simd, dw)


[[[[ 0.]
   [ 1.]
   [ 2.]
   [ 3.]
   [ 4.]
   [ 5.]]

  [[ 6.]
   [ 7.]
   [ 8.]
   [ 9.]
   [10.]
   [11.]]

  [[12.]
   [13.]
   [14.]
   [15.]
   [16.]
   [17.]]

  [[18.]
   [19.]
   [20.]
   [21.]
   [22.]
   [23.]]

  [[24.]
   [25.]
   [26.]
   [27.]
   [28.]
   [29.]]

  [[30.]
   [31.]
   [32.]
   [33.]
   [34.]
   [35.]]]]
[[[[ 0.  1.  6.  7. 12. 13.]
   [ 1.  2.  7.  8. 13. 14.]
   [ 2.  3.  8.  9. 14. 15.]
   [ 3.  4.  9. 10. 15. 16.]
   [ 4.  5. 10. 11. 16. 17.]]

  [[ 6.  7. 12. 13. 18. 19.]
   [ 7.  8. 13. 14. 19. 20.]
   [ 8.  9. 14. 15. 20. 21.]
   [ 9. 10. 15. 16. 21. 22.]
   [10. 11. 16. 17. 22. 23.]]

  [[12. 13. 18. 19. 24. 25.]
   [13. 14. 19. 20. 25. 26.]
   [14. 15. 20. 21. 26. 27.]
   [15. 16. 21. 22. 27. 28.]
   [16. 17. 22. 23. 28. 29.]]

  [[18. 19. 24. 25. 30. 31.]
   [19. 20. 25. 26. 31. 32.]
   [20. 21. 26. 27. 32. 33.]
   [21. 22. 27. 28. 33. 34.]
   [22. 23. 28. 29. 34. 35.]]]]
---
[[[[ 0.  1.  6.  7. 12. 13.]
   [ 1.  2.  7.  8. 13. 14.]
   [ 2.  3. 

In [3]:
from finn.util.visualization import showInNetron

#showInNetron("/tmp/test_convInputGenerator.onnx")

showInNetron("/tmp/test_convInputGenerator_compiled.onnx")


Serving '/tmp/test_convInputGenerator_compiled.onnx' at http://0.0.0.0:8081


In [11]:
from finn.custom_op.registry import getCustomOp
model = ModelWrapper("/tmp/test_convInputGenerator_compiled.onnx")
for n in model.graph.node:
    inst = getCustomOp(n)
    k = inst.get_nodeattr("ConvKernelDim")
    print("{}\n{}\n{}".format(k, type(k), np.shape(k)))
    
    k_h, k_w = inst.get_nodeattr("ConvKernelDim")
    print("{}\t{}".format(k_h, k_w))

[3, 2]
<class 'google.protobuf.pyext._message.RepeatedScalarContainer'>
(2,)
3	2


In [16]:
showInNetron("/tmp/test_convInputGenerator_im2col.onnx")

Stopping http://0.0.0.0:8081
Serving '/tmp/test_convInputGenerator_im2col.onnx' at http://0.0.0.0:8081


# Latest code

In [1]:
# Copyright (c) 2020, Xilinx
# All rights reserved.
#
# Redistribution and use in source and binary forms, with or without
# modification, are permitted provided that the following conditions are met:
#
# * Redistributions of source code must retain the above copyright notice, this
#   list of conditions and the following disclaimer.
#
# * Redistributions in binary form must reproduce the above copyright notice,
#   this list of conditions and the following disclaimer in the documentation
#   and/or other materials provided with the distribution.
#
# * Neither the name of FINN nor the names of its
#   contributors may be used to endorse or promote products derived from
#   this software without specific prior written permission.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
# AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
# IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
# DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE
# FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL
# DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR
# SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER
# CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY,
# OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
# OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

import pytest
import numpy as np

from onnx import TensorProto, helper

import finn.core.onnx_exec as oxe
from finn.core.datatype import DataType
from finn.core.modelwrapper import ModelWrapper
from finn.transformation.fpgadataflow.prepare_ip import PrepareIP
from finn.transformation.fpgadataflow.prepare_cppsim import PrepareCppSim
from finn.transformation.fpgadataflow.compile_cppsim import CompileCppSim
from finn.transformation.fpgadataflow.hlssynth_ip import HLSSynthIP
from finn.transformation.fpgadataflow.set_exec_mode import SetExecMode
from finn.transformation.fpgadataflow.prepare_rtlsim import PrepareRTLSim
from finn.transformation.general import GiveUniqueNodeNames
from finn.util.basic import gen_finn_dt_tensor

from finn.custom_op.registry import getCustomOp
from finn.analysis.fpgadataflow.exp_cycles_per_layer import exp_cycles_per_layer


def make_single_im2col_modelwrapper(k, ifm_ch, ifm_dim, ofm_dim, simd, stride, dilation, idt):
    odt = idt
    inp = helper.make_tensor_value_info(
        "inp", TensorProto.FLOAT, [1, ifm_dim, ifm_dim, ifm_ch]
    )
    outp = helper.make_tensor_value_info(
        "outp", TensorProto.FLOAT, [1, ofm_dim, ofm_dim, k * k * ifm_ch]
    )

    im2col_node = helper.make_node(
        "Im2Col",
        ["inp"],
        ["outp"],
        domain="finn.custom_op.general",
        backend="fpgadataflow",
        stride=[stride, stride],
        kernel_size=[k, k],
        input_shape=str((1, ifm_dim, ifm_dim, ifm_ch)),
        pad_amount=[0, 0, 0, 0],
        pad_value=0,
        dilations=[dilation, dilation]
    )
    graph = helper.make_graph(
        nodes=[im2col_node], name="im2col_graph", inputs=[inp], outputs=[outp]
    )

    model = helper.make_model(graph, producer_name="im2col-model")
    model = ModelWrapper(model)

    model.set_tensor_datatype("inp", idt)
    model.set_tensor_datatype("outp", odt)

    return model


def make_single_slidingwindow_modelwrapper(
    k, ifm_ch, ifm_dim, ofm_dim, simd, stride, dilation, idt, dw=0
):
    odt = idt
    inp = helper.make_tensor_value_info(
        "inp", TensorProto.FLOAT, [1, ifm_dim, ifm_dim, ifm_ch]
    )
    outp = helper.make_tensor_value_info(
        "outp", TensorProto.FLOAT, [1, ofm_dim, ofm_dim, k * k * ifm_ch]
    )

    SlidingWindow_node = helper.make_node(
        "ConvolutionInputGenerator",
        ["inp"],
        ["outp"],
        domain="finn.custom_op.fpgadataflow",
        backend="fpgadataflow",
        ConvKernelDim=[k, k],
        IFMChannels=ifm_ch,
        IFMDim=[ifm_dim, ifm_dim],
        OFMDim=[ofm_dim, ofm_dim],
        SIMD=simd,
        Stride=[stride, stride],
        Dilation=[dilation, dilation],
        inputDataType=idt.name,
        outputDataType=odt.name,
        depthwise=dw,
    )
    graph = helper.make_graph(
        nodes=[SlidingWindow_node],
        name="slidingwindow_graph",
        inputs=[inp],
        outputs=[outp],
    )

    model = helper.make_model(graph, producer_name="slidingwindow-model")
    model = ModelWrapper(model)

    model.set_tensor_datatype("inp", idt)
    model.set_tensor_datatype("outp", odt)

    return model


def prepare_inputs(input_tensor):
    return {"inp": input_tensor}


def test_fpgadataflow_slidingwindow(
    idt, k, ifm_dim, ifm_ch, stride, dilation, exec_mode, simd, dw
):
    ofm_dim = int(((ifm_dim - k) / stride) + 1)

    x = gen_finn_dt_tensor(idt, (1, ifm_dim, ifm_dim, ifm_ch))
    model = make_single_slidingwindow_modelwrapper(
        k, ifm_ch, ifm_dim, ofm_dim, simd, stride, dilation, idt, dw
    )

    if exec_mode == "cppsim":
        model = model.transform(SetExecMode("cppsim"))
        model = model.transform(PrepareCppSim())
        model = model.transform(CompileCppSim())
    elif exec_mode == "rtlsim":
        model = model.transform(SetExecMode("rtlsim"))
        model = model.transform(GiveUniqueNodeNames())
        model = model.transform(PrepareIP("xc7z020clg400-1", 5))
        model = model.transform(HLSSynthIP())
        model = model.transform(PrepareRTLSim())
    else:
        raise Exception("Unknown exec_mode in test_fpgadataflow_slidingwindow")

    # prepare input data
    input_dict = prepare_inputs(x)
    # execute model
    y_produced = oxe.execute_onnx(model, input_dict)["outp"]
    golden = make_single_im2col_modelwrapper(
        k, ifm_ch, ifm_dim, ofm_dim, simd, stride, dilation, idt
    )
    y_expected = oxe.execute_onnx(golden, input_dict)["outp"]
    if dw == 0:
        assert (y_produced == y_expected).all()
    else:
        y_expected = y_expected.reshape(
            1, ofm_dim, ofm_dim, k * k, ifm_ch // simd, simd
        )
        y_expected = y_expected.transpose(0, 1, 2, 4, 3, 5)
        y_expected = y_expected.reshape(1, ofm_dim, ofm_dim, ifm_ch * k * k)
        assert (y_produced == y_expected).all()

    if exec_mode == "rtlsim":
        node = model.get_nodes_by_op_type("ConvolutionInputGenerator")[0]
        inst = getCustomOp(node)
        cycles_rtlsim = inst.get_nodeattr("cycles_rtlsim")
        exp_cycles_dict = model.analysis(exp_cycles_per_layer)
        exp_cycles = exp_cycles_dict[node.name]
        assert np.isclose(exp_cycles, cycles_rtlsim, atol=10)
        assert exp_cycles != 0


In [2]:
idt = DataType.INT8
k = 2
ifm_dim = 6
ifm_ch = 2
stride = 1
dilation = 1
exec_mode = "cppsim"
simd = 1
dw = 1

test_fpgadataflow_slidingwindow(idt, k, ifm_dim, ifm_ch, stride, dilation, exec_mode, simd, dw)